In [ ]:
'''

Prototyping LLM EV tool to answer: is it cheaper to buy an EV or a gas car?

Input default is the above, but user can add additional information (I want a used car, I live in California, etc.)
This tool starts with answering from the broadest perspective and returns a 5 year cost projection
this tool then asks for more information to refine the answer


working

    DONE think through the more advanced graph
        maybe I want something that prepares the output for the user given all the information that has been gathered
            but if that agent is interacting with the user then isn't it the assistant in a way?
            can you tell the assistant how to output? is assitant getting overloaded
        I guess make a big prompt for assistant?
            then have a reflection node that can critique and offer suggestions

        so something like:
            user interacts with assistant
            based on user prompt, assistant comes up with a plan
            uses tools to get the parts
                can be sub assistants
            sends info back to assistant
                maybe this is a different agent/node
                maybe plan gets evaluated to make sure all info is good or complete
                assistant drafts response
            sends draft response to critic
                critic evalutes and offers suggestions
                sends back to assistant
            assistant sends to user
                
            different ways of doing some parts of this:    
                https://langchain-ai.github.io/langgraph/tutorials/rewoo/rewoo/
                    planner
                https://langchain-ai.github.io/langgraph/tutorials/reflection/reflection/
                    reflection

    TEST think through this graph a bit more
        main thing I am missing is most is user input back to assistant and want to loop on that
            but the examples are kind of sparse on that
            want the agent to persist and gather more info with each iteration
        maybe assume it works out of the box like that like example 1 https://langchain-ai.github.io/langgraph/tutorials/customer-support/customer-support/#part-1-zero-shot-agent
            and assume the memory persists
        maybe by keeping the thread the same?


    TEST simple tool usage
        have laid out all I can think of
        can do some simple ones to test out some things
        not sure how many of htem are needed and what the agent should be able to handle on its own
        could group and abstract the key tools
            then drill into them as things get better
        could try without the tool and run tests and see which are needed
            like this idea
            see what is needed first then add in the tools
        have a couple tools that are included for improving my skills
            do a search based tool for federal rebates / credits to start with
            everything else in the prompts for now

    DONE which tutorial do I want to base this work off of?
        leaning towards this one: https://langchain-ai.github.io/langgraph/tutorials/customer-support/customer-support/#part-4-specialized-workflows
            There are some other related oens that might be a bit simpler but the above should cover it
        Related and many steps but different formats
        https://langchain-ai.github.io/langgraph/tutorials/

    TEST write some basic code for minimal

    continue to organize the plan and notes

    research
        langraph
        tools
        similar ideas
        do city and towns have ev or car related taxes, fees, registrations etc

to do:

    more advanced graph initial idea:
        think through the more advanced graph
        maybe I want something that prepares the output for the user given all the information that has been gathered
            but if that agent is interacting with the user then isn't it the assistant in a way?
            can you tell the assistant how to output? is assitant getting overloaded
        I guess make a big prompt for assistant?
            then have a reflection node that can critique and offer suggestions

        so something like:
            user interacts with assistant
            based on user prompt, assistant comes up with a plan
            uses tools to get the parts
                can be sub assistants
            sends info back to assistant
                maybe this is a different agent/node
                maybe plan gets evaluated to make sure all info is good or complete
                assistant drafts response
            sends draft response to critic
                critic evalutes and offers suggestions
                sends back to assistant
            assistant sends to user
                
            different ways of doing some parts of this:    
                https://langchain-ai.github.io/langgraph/tutorials/rewoo/rewoo/
                    planner
                https://langchain-ai.github.io/langgraph/tutorials/reflection/reflection/
                    reflection

    after testing how the memory works through multiple interactions on the thread
        probably want something that goes over the history and stores key piece of info needed
        like the location, new or used, etc

    maybe initial version is lease versus buy for new EV

    improve my personal usage of LLM based coding tools

    graph types
        just ask the LLM
        LLM with tool usage
        LLM with sub agents and tool usage

    tool usage
        start small
        start broad
            can drill down into more tools as needed
        justify tool addition by showing it works better than the prompt
        
    langsmith
    test cases to judge the various iterations of the graphs


    move unfinished ideas to a consolidated place in the repo (from code notes and notes outside of repo)

    how to handle important discrete pieces of information
        ie location
        used or new etc
        model of car

    simple RAG
        for the cost estimates probably?

    continue to research langgraph docs and other creations people ahve made
    research other similar tools online

    reflection before passing to user
    user backa dn forther

    research best time period
        research how others like to lay out this info

    miles driven and how long the vehicle will likely last
        and years too
        might be people trade in more recently

    better config tavily search results

backlog:
    can probably store a lot of the data and answers
    non US locations
    can specifically ask for
        kw/hour cost
        per gallon gas price
    more advanced reasoning graphs
    Cost of capital and more advanced financial analysis

    advanced RAG of key information
        ie like the IRS website and how often should I updated it


resources
    https://langchain-ai.github.io/langgraph/

'''


### Constants

In [ ]:
import gradio as gr
import getpass
import os

from IPython.display import Image, display

from langchain_openai import OpenAI, ChatOpenAI

from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig

from pydantic import BaseModel, Field

from typing import Annotated, Literal, Optional

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages

from langchain_core.tools import tool

from typing import Literal

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition

import pgeocode

import uuid

In [ ]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")
_set_env("LANGSMITH_API_KEY")
_set_env("TAVILY_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "EV Prototype v0.4"



In [ ]:
testing_questions_list = [
    'What is the cost of owning a new car?',
    #'What is the cost of owning an electric vehicle in California?',
]

In [ ]:
def display_graph(graph):
    try:
        display(Image(graph.get_graph().draw_mermaid_png()))
    except Exception:
        # This requires some extra dependencies and is optional
        pass


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

### v0.01 Minimal LLM Based
* Baseline
* What if the LLM is asked straight up to answer the question

In [ ]:
'''
Tried this prompt to copilot. terrible response:
# define the langchain graph
# the only node should be the LLM node
# base the graph off of this tutorial
# https://langchain-ai.github.io/langgraph/tutorials/customer-support/customer-support/


'''

In [ ]:


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    user_info: str
    # dialog_state: Annotated[
    #     list[
    #         Literal[
    #             "assistant",
    #             "update_flight",
    #             "book_car_rental",
    #             "book_hotel",
    #             "book_excursion",
    #         ]
    #     ],
    #     update_dialog_stack,
    # ]

In [ ]:
class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            result = self.runnable.invoke(state)

            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}


llm = ChatOpenAI(model_name="gpt-3.5-turbo")


ev_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant that helps gather information for people deciding to buy or lease an electric vehicle or a gas car."
            "Help them compare the costs of buying or leasing an electric vehicle or a gas car."
            "The fundamental question you are trying to answer is whether it is cheaper to buy an electric vehicle or a gas car."
            "If the user does not provide detailed information, answer the question generally."
            "If the user does provide detailed information, use that information to provider a better answer."
            "Try to ask the user for more information after you provider your answer."
            "Some examples of key pieces of information to ask include:\n"
            " - Location\n"
            " - Used or new\n"
            " - Type of car they are interested in (sedan, luxury, compact-SUV, mid-size truck, etc.) \n"
        ),
        ("placeholder", "{messages}"),
    ]
)

ev_assistant_runnable = ev_prompt | llm

builder = StateGraph(State)

builder.add_node("assistant", Assistant(ev_assistant_runnable))
# builder.set_entry_point("assistant")
builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

memory = MemorySaver()
baseline_agent_graph = builder.compile(
    checkpoint=memory,
)

display_graph(baseline_agent_graph)

In [ ]:
thread_id = str(uuid.uuid4())

config = {
    "configurable": {

        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}

_printed = set()
for question in testing_questions_list:
    events = baseline_agent_graph.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)


### v0.1 LLM and Tools
* can do LLM and graph mostly based on the above
* will need tool selection part
* need tool nodes
* need tool functions


#### Possible tools
* cost estimate based tools
    * tool: estimate various expenses. overall and breakdown when asked
        * By subject and by time period (like year)
        * start general tool. can then go into better tools as things get better
        * car ownership expenses
            * insurance
                * liability
                * gap
            * registration
            * gas
            * electricity
                * home charging
                * public charging
            * maintenance
            * state and local fees & taxes
            * federal fees and taxes (not applicable I don't think?)
            * depreciation
            * resale value
            * Lesser fees / later
                * licensing
                * title fees
                * dealer add ons like extended warranty etc
        * Acquisition related fees
            * dealer fees
            * down payment
            * first monthly payment
            * financing fee
            * later
                * warranty/add ons
            * lease related
                * lease money down
                * lease monthly
                * excess mileage on lease
                * wear and tear fees on lease
                * end of lease vee
                * vehicle return
                * acquisition fee
                * lease downpayment fee
* search based tools
    * vehicle prices
        * new
        * used
        * lease deals
    * insurance quotes
    * latest federal rules for rebates
    * latest state rules for rebates
    * latest state /local fees
    * latests federal fees
    * cost of electricity by state / zip
    * cost of gas by state / zip
    * trade in tool
    * location: want city and state
        can ask specifically or ask for zip
* math based / abstraction tools
    * make a grid by time period
    * make a grid with subect break out
    * basic addition between outputs of various things
    * handling the outputs of the LLM
* finance tools
    * loan calculator
    * alternative investment
* time windwo to consider
    * if not user given and miles per year not given then do an overall estimate up to some number
        * have to research this
* financing vs. down payment vs. how much to put down

        


In [ ]:
@tool
def get_state_and_city_from_zip_code(zip_code: str):
    """Get the United States state and city from a five digit zip code
    
    Returns:
        state_name: str: The state name. 'unknown' if the zip code is invalid
        place_name: str: The city name. 'unknown' if the zip code is invalid
    """
    unknown_value = 'unknown'

    if zip_code is None or type(zip_code) is not str or len(zip_code) != 5:
        state_name = unknown_value
        place_name = unknown_value

    else:
        nomi = pgeocode.Nominatim('us')

        results_df = nomi.query_postal_code(zip_code)

        if results_df.shape[0] > 0:
            state_name = results_df.state_name
            place_name = results_df.place_name
        else:
            state_name = unknown_value
            place_name = unknown_value

    return state_name, place_name


#### Graph

In [ ]:


class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            result = self.runnable.invoke(state)

            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}


llm = ChatOpenAI(model_name="gpt-3.5-turbo")


ev_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant that helps gather information for people deciding to buy or lease an electric vehicle or a gas car. "
            "Help them compare the costs of buying or leasing an electric vehicle or a gas car. "
            "The fundamental question you are trying to answer is whether it is cheaper to buy an electric vehicle or a gas car. "
            "You have access to the Internt through the Tavily API search tool. "
            "When using the search tool, be persistent. Expand your search if the first search returns no results. "
            "If a search comes up empty, expand your search before giving up. "
            "If the user does not provide detailed information, answer the question generally. "
            "If the user does provide detailed information, use that information to provider a better answer. "
            "Try to ask the user for more information after you provider your answer. " 
            "Some examples of key pieces of information to ask include:\n"
            " - Location\n"
            " - Used or new\n"
            " - Type of car they are interested in (sedan, luxury, compact-SUV, mid-size truck, etc.) \n"
        ),
        ("placeholder", "{messages}"),
    ]
)

test_tool_list = [
    TavilySearchResults(max_results=1),
    get_state_and_city_from_zip_code,
]

ev_assistant_runnable = ev_prompt | llm.bind_tools(test_tool_list)

builder = StateGraph(State)

builder.add_node("assistant", Assistant(ev_assistant_runnable))
# builder.set_entry_point("assistant")
builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

memory = MemorySaver()
baseline_agent_graph = builder.compile(
    checkpoint=memory,
)

display_graph(baseline_agent_graph)

#### Execution

In [ ]:
thread_id = str(uuid.uuid4())

config = {
    "configurable": {

        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}

_printed = set()
for question in testing_questions_list:
    events = baseline_agent_graph.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)

### Supervisor, sub-agents, and Tools